In [1]:
import sys
sys.executable

'/bin/python3'

In [2]:
# !echo -e "yfinance\nsmartapi-python\npyotp\nlogzero\nstreamlit\nFlask\ndatetime\npandas\nnumpy\nfyers-apiv3\nwebsockets\nwebsocket-client" > requirements.txt
# !pip3 install -r re*

In [3]:
!pip3 install fyers-apiv3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 248.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 367.7 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 810.1 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 415.1 kB/s eta 0:00:000:00:01 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 KB 576.6 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 KB 852.3 kB/s eta 0:00:001m933.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 KB 915.4 kB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 KB 1.4 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 508.6 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━

In [4]:
#gstatparth@gmail.com
#xp12325
#https://myapi.fyers.in/dashboard
redircet_url = 'http://127.0.0.1:8081'
client_id="XD01RU06CS-100"
secret_key="EP5DDWV20I"
FY_ID="XP12325"
TOTP_KEY="Q2HC7F57FHMHPRT2VRLPRWA4ORWPK34E"
PIN="8894"
grant_type="authorization_code"
response_type="code"




In [5]:
import os
import requests, time, base64, struct, hmac
from fyers_apiv3 import fyersModel
from urllib.parse import urlparse, parse_qs #https://www.youtube.com/watch?v=UHX0jwpw0_E
pin = '8894'
class FyesApp:
    def __init__(self) -> None:
        self.__username= 'XP12325' 
        self.__totp_key='Q2HC7F57FHMHPRT2VRLPRWA4ORWPK34E'
        self.__pin='8894'
        self.__client_id="M6EQ9SEMLM-100"
        self.__secret_key="22NRKYLP40"
        self.__redirect_uri='http://127.0.0.1:8081'
        self.__access_token=None

    def enable_app(self):
        appSession= fyersModel.SessionModel(
            client_id= self.__client_id,
            redirect_uri= self.__redirect_uri,
            response_type= 'code',
            state= 'state',
            secret_key=self.__secret_key,
            grant_type='authorization_code'
            
        )
        return appSession.generate_authcode()

    #private function for authenticator app function for authentacation using totp
    def __totp(self, key, time_step=30, digits=6, digest="sha1"):
        key = base64.b32decode(key.upper() + "=" * ((8 - len(key)) % 8))
        counter = struct.pack(">Q", int(time.time() / time_step))
        mac = hmac.new(key, counter, digest).digest()
        offset = mac[-1] & 0x0F
        binary = struct.unpack(">L", mac[offset : offset + 4])[0] & 0x7FFFFFFF
        return str(binary)[-digits:].zfill(digits)

    def get_token(self, refresh=False):
        if self.__access_token == None and refresh !=False:
            return #?

        headers = {
            "Accept": "application/json",
            "Accept-Language": "en-US,en;q=0.9",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
        }

        s = requests.Session()
        s.headers.update(headers)

        data1 = f'{{"fy_id":"{base64.b64encode(f"{self.__username}".encode()).decode()}","app_id":"2"}}'
        r1 = s.post("https://api-t2.fyers.in/vagator/v2/send_login_otp_v2", data=data1)

        request_key = r1.json()["request_key"]
        data2 = f'{{"request_key":"{request_key}","otp":{self.__totp(self.__totp_key)}}}'
        r2 = s.post("https://api-t2.fyers.in/vagator/v2/verify_otp", data=data2)
        assert r2.status_code == 200, f"Error in r2:\n {r2.text}"

        request_key = r2.json()["request_key"]
        data3 = f'{{"request_key":"{request_key}","identity_type":"pin","identifier":"{base64.b64encode(f"{pin}".encode()).decode()}"}}'
        r3 = s.post("https://api-t2.fyers.in/vagator/v2/verify_pin_v2", data=data3)
        assert r3.status_code == 200, f"Error in r3:\n {r3.json()}"

        headers = {"authorization": f"Bearer {r3.json()['data']['access_token']}", "content-type": "application/json; charset=UTF-8"}
        data4 = f'{{"fyers_id":"{self.__username}","app_id":"{self.__client_id[:-4]}","redirect_uri":"{self.__redirect_uri}","appType":"100","code_challenge":"","state":"abcdefg","scope":"","nonce":"","response_type":"code","create_cookie":true}}'
        r4 = s.post("https://api.fyers.in/api/v2/token", headers=headers, data=data4)
        assert r4.status_code == 308, f"Error in r4:\n {r4.json()}"

        parsed = urlparse(r4.json()["Url"])
        auth_code = parse_qs(parsed.query)["auth_code"][0]

        session = fyersModel.SessionModel(
            client_id=self.__client_id, 
            secret_key=self.__secret_key, 
            redirect_uri=self.__redirect_uri, 
            response_type="code", 
            grant_type="authorization_code"
            )

        session.set_token(auth_code)
        response = session.generate_token()
        self.__access_token =  response["access_token"]
        return self.__access_token

app = FyesApp()
print(app.enable_app())
# app.get_token()
# print(app.__access_token())

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=M6EQ9SEMLM-100&redirect_uri=http%3A%2F%2F127.0.0.1%3A8081&response_type=code&state=state


In [6]:
print(app.get_token())

eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MzEyMzQ1ODAsImV4cCI6MTczMTI4NTA0MCwibmJmIjoxNzMxMjM0NTgwLCJhdWQiOlsieDowIiwieDoxIiwiZDoxIiwiZDoyIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbk1Jc1V1YjREYldfQjFBV2xXMGNCU3lzYWdfOVBUb0hTdVdBcHJwaXNpTEdzWUhaNEF6aGJqWGNOWG0yMmFGYi1TSWVNRkI5MjFMMzJFN3c0LTNNSWRSdy1qQ2FjbEFkTW5UY1ktQm1xWDlIMWFURT0iLCJkaXNwbGF5X25hbWUiOiJQQVJUSCBST1lBTCIsIm9tcyI6IksxIiwiaHNtX2tleSI6ImYyOTA3Njg3N2YwMTQ2M2YzY2RlODNiZDI1N2Q4ODcwYWY3Nzc3YTE3N2JiOTRkNDc4OTczZWRmIiwiZnlfaWQiOiJYUDEyMzI1IiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.iks2y5rc4Nhg49OSfvx76iYVJ0m8kCDfuIdOjHuKcUo


In [7]:
at = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MzEyMzQ1ODAsImV4cCI6MTczMTI4NTA0MCwibmJmIjoxNzMxMjM0NTgwLCJhdWQiOlsieDowIiwieDoxIiwiZDoxIiwiZDoyIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbk1Jc1V1YjREYldfQjFBV2xXMGNCU3lzYWdfOVBUb0hTdVdBcHJwaXNpTEdzWUhaNEF6aGJqWGNOWG0yMmFGYi1TSWVNRkI5MjFMMzJFN3c0LTNNSWRSdy1qQ2FjbEFkTW5UY1ktQm1xWDlIMWFURT0iLCJkaXNwbGF5X25hbWUiOiJQQVJUSCBST1lBTCIsIm9tcyI6IksxIiwiaHNtX2tleSI6ImYyOTA3Njg3N2YwMTQ2M2YzY2RlODNiZDI1N2Q4ODcwYWY3Nzc3YTE3N2JiOTRkNDc4OTczZWRmIiwiZnlfaWQiOiJYUDEyMzI1IiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.iks2y5rc4Nhg49OSfvx76iYVJ0m8kCDfuIdOjHuKcUo"

In [8]:
from fyers_apiv3 import fyersModel

client_id = "M6EQ9SEMLM-100"

# Initialize the FyersModel instance with your client_id, access_token, and enable async mode
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=at, log_path="./")

# Make a request to get the user profile information
response = fyers.get_profile()

# Print the response received from the Fyers API
print(response)




{'s': 'ok', 'code': 200, 'message': '', 'data': {'fy_id': 'XP12325', 'name': 'PARTH ROYAL', 'image': None, 'display_name': None, 'pin_change_date': '05-12-2021 17:10:44', 'email_id': 'gstatparth@gmail.com', 'pwd_change_date': '27-01-2022 11:39:41', 'PAN': '---------', 'mobile_number': '9418037413', 'totp': True, 'pwd_to_expire': -867}}
